In [1]:
## make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path

file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))

%load_ext autoreload
%autoreload

import pandas as pd
import numpy as np
from dataclasses import dataclass


from pa_lib.file import (
    project_dir,
    load_bin,
    load_pickle,
    load_xlsx,
    store_bin,
)
from pa_lib.data import (
    as_dtype,
    dtFactor,
    lookup,
    desc_col,
    chi2_expected,
    clean_up_categoricals,
)
from pa_lib.util import cap_words, collect, value, normalize_rows, as_percent, flatten

# display long columns completely, show more rows
pd.set_option("display.max_colwidth", 200)
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

# Load data sets

In [2]:
with project_dir("axinova"):
    ax_data = load_bin("ax_data.feather")
    spr_data = load_pickle("spr_data.pkl")
    time_codes = load_pickle("time_code_ratios.pkl")
    station_codes = load_pickle("station_code_ratios.pkl")
    global_codes = load_pickle("global_code_ratios.pkl")
    population_codes = load_pickle("population_ratios.pkl")

15:22:50 [INFO] Started loading binary file ...
15:22:50 [INFO] Reading from file C:\Users\kpf\data\axinova\ax_data.feather
15:22:51 [INFO] ... finished loading binary file in 1.1s (1.62s CPU)
15:22:51 [INFO] Started loading pickle file ...
15:22:51 [INFO] Reading from file C:\Users\kpf\data\axinova\spr_data.pkl
15:22:51 [INFO] ... finished loading pickle file in 0.03s (0.02s CPU)
15:22:51 [INFO] Started loading pickle file ...
15:22:51 [INFO] Reading from file C:\Users\kpf\data\axinova\time_code_ratios.pkl
15:22:51 [INFO] ... finished loading pickle file in 0.03s (0.0s CPU)
15:22:51 [INFO] Started loading pickle file ...
15:22:51 [INFO] Reading from file C:\Users\kpf\data\axinova\station_code_ratios.pkl
15:22:51 [INFO] ... finished loading pickle file in 0.0s (0.0s CPU)
15:22:51 [INFO] Started loading pickle file ...
15:22:51 [INFO] Reading from file C:\Users\kpf\data\axinova\global_code_ratios.pkl
15:22:51 [INFO] ... finished loading pickle file in 0.0s (0.02s CPU)
15:22:51 [INFO] St

## Select Axinova data by any column(s)

In [3]:
def _check_selection(data, selection, allowed_columns):
    allowed_values = {}
    for column in allowed_columns:
        allowed_values[column] = data[column].cat.categories
    if set(selection.keys()) - set(allowed_columns) != set():
        raise NameError(f"Unknown column name in selection: {selection.keys()}")
    clean_selection = {}
    for column in allowed_columns:
        if column in selection:
            col_values = list(flatten(selection[column]))
            if set(col_values) - set(allowed_values[column]) != set():
                raise ValueError(
                    f"Illegal value(s) in parameter {column}: {col_values}"
                )
            clean_selection[column] = col_values
        else:
            clean_selection[column] = None
    return clean_selection


def select_data(all_data, **selection):
    select_columns = (
        "DayOfWeek Station Variable Month TimeSlot Hour "
        + "Time TimeSlot_cat StationSprache".split()
    )
    selection = _check_selection(all_data, selection, allowed_columns=select_columns)
    row_mask = pd.Series([True] * all_data.shape[0])
    for col in select_columns:
        if selection[col] is not None:
            row_mask &= all_data[col].isin(selection[col])
    return all_data.loc[row_mask].pipe(clean_up_categoricals).reset_index(drop=True)

# Look up code ratios from Axinova data

In [14]:
RatioTable = pd.DataFrame


@dataclass
class Ratios:
    actual: RatioTable
    expected: RatioTable
    sd: RatioTable = None

In [15]:
def ax_population_ratios(variable: str, percent: bool = False) -> RatioTable:
    ratios = population_codes.query("Variable == @variable").pivot_table(
        values="Pop_Ratio", index="Variable", columns="Code"
    )
    return as_percent(ratios) if percent else ratios


def ax_global_ratios(variable: str, percent: bool = False) -> RatioTable:
    ratios = global_codes.query("Variable == @variable").pivot_table(
        values="Ratio", index="Variable", columns="Code"
    )
    return as_percent(ratios) if percent else ratios


def ax_station_ratios(variable: str, percent: bool = False) -> Ratios:
    actual_ratios = station_codes.query("Variable == @variable").pivot_table(
        values="Ratio", index="Station", columns="Code", fill_value=0
    )
    expected_ratios = ax_global_ratios(variable)
    if percent:
        result = Ratios(
            actual=as_percent(actual_ratios), expected=as_percent(expected_ratios)
        )
    else:
        result = Ratios(actual=actual_ratios, expected=expected_ratios)
    return result

In [63]:
def ax_ratios(
    variable: str,
    stations: str,
    reference: str = "all_stations",
    time_scale: str = "Hour",
    percent: bool = False,
) -> Ratios:
    subset = ax_data.loc[
        ax_data.Station.isin(flatten(stations)) & (ax_data.Variable == variable)
    ]
    full_index = [
        (weekday, time)
        for weekday in ax_data["DayOfWeek"].cat.categories
        for time in ax_data[time_scale].cat.categories
    ]
    actual_counts = subset.pivot_table(
        values="Value",
        index=["DayOfWeek", time_scale],
        columns="Code",
        fill_value=0,
        aggfunc="sum",
    )
    actual_counts_sd_ratios = (
        (np.sqrt(actual_counts) / actual_counts)
        .fillna(0)
        .reindex(full_index, fill_value=0)
    )
    actual_ratios = normalize_rows(actual_counts).reindex(full_index, fill_value=0)

    if reference == "all_stations":
        expected_ratios = (
            time_codes[time_scale]
            .query("Variable == @variable")
            .pivot_table(
                values="Ratio",
                index=["DayOfWeek", time_scale],
                columns="Code",
                fill_value=0,
            )
        )
    elif reference == "station":
        expected_counts = chi2_expected(actual_counts)
        expected_ratios = normalize_rows(expected_counts).reindex(
            full_index, fill_value=0
        )
    else:
        raise ValueError(
            "Parameter 'reference' must be one of "
            + f"('station', 'all_stations'), was '{reference}'"
        )

    if percent:
        result = Ratios(
            actual=as_percent(actual_ratios),
            expected=as_percent(expected_ratios),
            sd=as_percent(actual_counts_sd_ratios),
        )
    else:
        result = Ratios(
            actual=actual_ratios, expected=expected_ratios, sd=actual_counts_sd_ratios
        )
    return result

# Look up SPR+ data split by variable

In [17]:
def spr_split(
    stations,
    variable,
    reference="station",
    time_scale="Hour",
    type="abs",
    incl_totals=True,
    decimals=1,
):
    if type not in ["abs", "rel"]:
        raise ValueError(
            f"Parameter 'type' must be one of ('abs', 'rel'), was '{type}'"
        )
    if reference not in ["station", "all_stations"]:
        raise ValueError(
            "Parameter 'reference' must be one of "
            + f"('station', 'all_stations'), was '{reference}'"
        )

    spr_counts = (
        spr_data.loc[spr_data.Station.isin(flatten(stations))]
        .groupby(["DayOfWeek", time_scale])[["Total"]]
        .agg("sum")
    )
    spr_count_sd_ratios = (np.sqrt(spr_counts) / spr_counts).fillna(0)

    ratios = ax_ratios(
        stations=stations,
        variable=variable,
        reference=reference,
        time_scale=time_scale,
        percent=False,
    )
    if type == "abs":
        code_ratios = ratios.actual
    elif type == "rel":
        code_ratios = ratios.actual - ratios.expected
    split_counts = code_ratios.mul(spr_counts.Total, axis="index").round(decimals)

    sd_ratios = np.sqrt(
        (ratios.sd ** 2).add((spr_count_sd_ratios ** 2).values, axis="index")
    )

    if incl_totals:
        split_counts.set_axis(split_counts.columns.to_list(), axis="columns", inplace=True)
        split_counts["Total"] = spr_counts.round(decimals)
        sd_ratios.set_axis(sd_ratios.columns.to_list(), axis="columns", inplace=True)
        sd_ratios["Total"] = spr_count_sd_ratios.values
    if decimals == 0:
        split_counts = split_counts.astype("int")

    return split_counts, sd_ratios

# Tests

## Station ratios

In [18]:
with value(ax_station_ratios(variable="g_220", percent=True)) as autobesitz:
    display(autobesitz.actual)
    display(autobesitz.expected)
    display(autobesitz.actual.sub(autobesitz.expected.values, axis="columns"))

Code,1 Auto,2+ Autos,Keines
Station,,,
Aarau,43,30,27
Basel SBB,44,20,36
Bellinzona,52,38,10
Bern,43,28,28
Biel/Bienne,50,25,25
Brig,51,40,9
Chur,48,35,17
Fribourg,31,56,14
Genève Aéroport,41,49,10


Code,1 Auto,2+ Autos,Keines
Variable,,,
g_220,45,29,26


Code,1 Auto,2+ Autos,Keines
Station,,,
Aarau,-2,1,1
Basel SBB,-1,-9,10
Bellinzona,7,9,-16
Bern,-2,-1,2
Biel/Bienne,5,-4,-1
Brig,6,11,-17
Chur,3,6,-9
Fribourg,-14,27,-12
Genève Aéroport,-4,20,-16


## Time ratios for one station

In [19]:
with value(ax_ratios(stations="Lausanne", variable="g_220")) as lausanne_auto:
    display(lausanne_auto.actual)
    display(lausanne_auto.expected)
    display(lausanne_auto.actual - lausanne_auto.expected)

Code              1 Auto  2+ Autos    Keines
DayOfWeek Hour                              
Monday    00    0.312211  0.600001  0.087787
          01    1.000000  0.000000  0.000000
          02    0.000000  0.000000  0.000000
          03    0.581119  0.000000  0.418881
          04    0.380052  0.000000  0.619948
          05    0.101995  0.729447  0.168557
          06    0.455790  0.143810  0.400400
          07    0.414627  0.425143  0.160231
          08    0.576622  0.278330  0.145048
          09    0.592880  0.309610  0.097510
          10    0.477487  0.402928  0.119585
          11    0.538332  0.304849  0.156819
          12    0.616668  0.180973  0.202359
          13    0.604475  0.314736  0.080789
          14    0.395047  0.413726  0.191227
          15    0.433369  0.411901  0.154730
          16    0.406679  0.431233  0.162088
          17    0.418946  0.437566  0.143488
          18    0.413588  0.370438  0.215974
          19    0.442323  0.174212  0.383465
          20    0.526554  0.309764  0.163682
          21    0.543837  0.295797  0.160366
          22    0.490854  0.147185  0.361961
          23    0.418856  0.234218  0.346926
Tuesday   00    0.113605  0.480522  0.405874
          01    0.000000  0.348870  0.651130
          02    0.000000  0.000000  0.000000
          03    0.000000  0.000000  0.000000
          04    0.000000  0.000000  0.000000
          05    0.154251  0.845749  0.000000
          06    0.419888  0.114871  0.465241
          07    0.577460  0.284159  0.138381
          08    0.625329  0.222935  0.151736
          09    0.361894  0.416258  0.221848
          10    0.572257  0.279723  0.148020
          11    0.607328  0.314228  0.078445
          12    0.591319  0.316871  0.091810
          13    0.550939  0.250173  0.198888
          14    0.420224  0.456244  0.123533
          15    0.442870  0.411823  0.145307
          16    0.369764  0.387674  0.242562
          17    0.449946  0.401211  0.148842
          18    0.490498  0.262222  0.247281
          19    0.399974  0.263110  0.336916
          20    0.289134  0.437553  0.273314
          21    0.254541  0.379299  0.366161
          22    0.357687  0.309989  0.332323
          23    0.581290  0.179499  0.239211
Wednesday 00    0.623126  0.263355  0.113519
          01    0.208043  0.546500  0.245457
          02    0.000000  1.000000  0.000000
          03    0.000000  0.000000  1.000000
          04    0.000000  0.000000  0.000000
          05    0.299848  0.651801  0.048351
          06    0.369472  0.159110  0.471417
          07    0.509278  0.340491  0.150232
          08    0.590114  0.337018  0.072868
          09    0.382259  0.466019  0.151721
          10    0.377912  0.455357  0.166731
          11    0.394669  0.541450  0.063881
          12    0.595034  0.218337  0.186629
          13    0.569747  0.309292  0.120961
          14    0.453393  0.337867  0.208740
          15    0.329974  0.544719  0.125307
          16    0.373862  0.403866  0.222271
          17    0.464287  0.370080  0.165634
          18    0.460964  0.242585  0.296452
          19    0.379379  0.282101  0.338520
          20    0.420005  0.368363  0.211632
          21    0.348701  0.414411  0.236887
          22    0.256763  0.403235  0.340002
          23    0.293267  0.284189  0.422544
Thursday  00    0.496029  0.390298  0.113672
          01    0.608040  0.168125  0.223835
          02    0.000000  0.000000  0.000000
          03    0.000000  0.000000  0.000000
          04    1.000000  0.000000  0.000000
          05    0.063307  0.857580  0.079113
          06    0.420381  0.188800  0.390819
          07    0.477858  0.358015  0.164128
          08    0.639473  0.279896  0.080631
          09    0.511990  0.404198  0.083811
          10    0.429781  0.481817  0.088402
          11    0.499171  0.302363  0.198466
          12    0.639242  0.235416  0.125342
          13    0.493861  0.388029  0.118110
          14    0.434190  0.404615  0.16

Code              1 Auto  2+ Autos    Keines
DayOfWeek Hour                              
Monday    00    0.379786  0.316787  0.303428
          01    0.505276  0.251623  0.243101
          02    0.374364  0.060254  0.565383
          03    0.486027  0.395825  0.118148
          04    0.285555  0.409515  0.304931
          05    0.411548  0.293442  0.295009
          06    0.473889  0.231063  0.295048
          07    0.485756  0.303293  0.210951
          08    0.473161  0.300682  0.226156
          09    0.431334  0.294713  0.273952
          10    0.416444  0.316025  0.267531
          11    0.418915  0.314254  0.266831
          12    0.442496  0.282251  0.275253
          13    0.420893  0.308341  0.270766
          14    0.446500  0.299036  0.254464
          15    0.458426  0.285930  0.255644
          16    0.474955  0.304906  0.220139
          17    0.502548  0.288892  0.208560
          18    0.472755  0.240098  0.287147
          19    0.456171  0.242598  0.301231
          20    0.433826  0.270623  0.295551
          21    0.406813  0.233773  0.359414
          22    0.473902  0.202478  0.323621
          23    0.484514  0.241980  0.273506
Tuesday   00    0.366221  0.417344  0.216435
          01    0.576078  0.173811  0.250111
          02    0.456756  0.381457  0.161787
          03    0.950507  0.049493  0.000000
          04    0.252567  0.578466  0.168968
          05    0.313339  0.334682  0.351979
          06    0.447504  0.273974  0.278522
          07    0.499753  0.291496  0.208752
          08    0.508946  0.281311  0.209743
          09    0.416510  0.349783  0.233707
          10    0.401771  0.326560  0.271669
          11    0.433715  0.315540  0.250746
          12    0.455890  0.315558  0.228552
          13    0.442159  0.318962  0.238879
          14    0.453551  0.339946  0.206503
          15    0.441810  0.290321  0.267869
          16    0.477069  0.320641  0.202290
          17    0.474824  0.301256  0.223920
          18    0.470500  0.234037  0.295463
          19    0.459756  0.262799  0.277445
          20    0.431106  0.297967  0.270927
          21    0.426353  0.269335  0.304312
          22    0.396796  0.263915  0.339289
          23    0.513284  0.248932  0.237784
Wednesday 00    0.455195  0.331925  0.212880
          01    0.358003  0.284824  0.357172
          02    0.358501  0.182853  0.458645
          03    0.000000  0.673257  0.326743
          04    0.372727  0.388252  0.239021
          05    0.288878  0.293321  0.417801
          06    0.460823  0.269780  0.269397
          07    0.502400  0.282044  0.215556
          08    0.479994  0.295433  0.224573
          09    0.421216  0.339932  0.238851
          10    0.404577  0.310582  0.284841
          11    0.427140  0.345842  0.227018
          12    0.433377  0.305243  0.261380
          13    0.426852  0.325979  0.247169
          14    0.446749  0.322384  0.230867
          15    0.434929  0.327478  0.237593
          16    0.462679  0.306264  0.231057
          17    0.493546  0.278523  0.227931
          18    0.450820  0.254891  0.294290
          19    0.434221  0.272131  0.293648
          20    0.429969  0.309908  0.260124
          21    0.438970  0.265934  0.295096
          22    0.441984  0.287730  0.270286
          23    0.400625  0.292503  0.306873
Thursday  00    0.353429  0.369613  0.276958
          01    0.302990  0.263444  0.433567
          02    0.316923  0.336055  0.347022
          03    0.423996  0.243207  0.332798
          04    0.514300  0.290016  0.195684
          05    0.350982  0.282728  0.366290
          06    0.505294  0.260533  0.234173
          07    0.513353  0.272503  0.214144
          08    0.492358  0.287497  0.220145
          09    0.448995  0.335357  0.215648
          10    0.455830  0.307173  0.236996
          11    0.443228  0.305746  0.251026
          12    0.444632  0.319084  0.236284
          13    0.468341  0.300388  0.231271
          14    0.476870  0.292025  0.23

Code              1 Auto  2+ Autos    Keines
DayOfWeek Hour                              
Monday    00   -0.067574  0.283214 -0.215640
          01    0.494724 -0.251623 -0.243101
          02   -0.374364 -0.060254 -0.565383
          03    0.095092 -0.395825  0.300733
          04    0.094497 -0.409515  0.315017
          05   -0.309553  0.436005 -0.126452
          06   -0.018100 -0.087253  0.105352
          07   -0.071129  0.121850 -0.050720
          08    0.103461 -0.022352 -0.081108
          09    0.161545  0.014897 -0.176442
          10    0.061043  0.086904 -0.147947
          11    0.119417 -0.009405 -0.110012
          12    0.174172 -0.101278 -0.072894
          13    0.183582  0.006395 -0.189977
          14   -0.051453  0.114690 -0.063236
          15   -0.025057  0.125971 -0.100914
          16   -0.068275  0.126326 -0.058051
          17   -0.083601  0.148674 -0.065072
          18   -0.059167  0.130340 -0.071173
          19   -0.013848 -0.068386  0.082234
          20    0.092728  0.039141 -0.131869
          21    0.137024  0.062024 -0.199048
          22    0.016953 -0.055293  0.038340
          23   -0.065658 -0.007762  0.073420
Tuesday   00   -0.252617  0.063178  0.189438
          01   -0.576078  0.175060  0.401019
          02   -0.456756 -0.381457 -0.161787
          03   -0.950507 -0.049493  0.000000
          04   -0.252567 -0.578466 -0.168968
          05   -0.159088  0.511067 -0.351979
          06   -0.027616 -0.159104  0.186719
          07    0.077708 -0.007337 -0.070371
          08    0.116383 -0.058376 -0.058007
          09   -0.054616  0.066475 -0.011859
          10    0.170487 -0.046837 -0.123649
          11    0.173613 -0.001312 -0.172301
          12    0.135429  0.001313 -0.136742
          13    0.108780 -0.068789 -0.039991
          14   -0.033327  0.116298 -0.082971
          15    0.001060  0.121501 -0.122561
          16   -0.107305  0.067033  0.040272
          17   -0.024878  0.099955 -0.075078
          18    0.019998  0.028184 -0.048182
          19   -0.059782  0.000311  0.059471
          20   -0.141973  0.139586  0.002386
          21   -0.171812  0.109963  0.061849
          22   -0.039109  0.046075 -0.006965
          23    0.068007 -0.069433  0.001427
Wednesday 00    0.167931 -0.068569 -0.099361
          01   -0.149961  0.261676 -0.111715
          02   -0.358501  0.817147 -0.458645
          03    0.000000 -0.673257  0.673257
          04   -0.372727 -0.388252 -0.239021
          05    0.010970  0.358480 -0.369450
          06   -0.091351 -0.110670  0.202020
          07    0.006878  0.058447 -0.065325
          08    0.110121  0.041584 -0.151705
          09   -0.038957  0.126087 -0.087130
          10   -0.026665  0.144775 -0.118110
          11   -0.032471  0.195608 -0.163137
          12    0.161658 -0.086906 -0.074752
          13    0.142895 -0.016688 -0.126207
          14    0.006644  0.015483 -0.022127
          15   -0.104955  0.217240 -0.112286
          16   -0.088817  0.097602 -0.008785
          17   -0.029259  0.091557 -0.062297
          18    0.010144 -0.012306  0.002162
          19   -0.054842  0.009970  0.044872
          20   -0.009963  0.058455 -0.048492
          21   -0.090269  0.148477 -0.058208
          22   -0.185221  0.115505  0.069716
          23   -0.107358 -0.008314  0.115672
Thursday  00    0.142600  0.020686 -0.163286
          01    0.305050 -0.095318 -0.209732
          02   -0.316923 -0.336055 -0.347022
          03   -0.423996 -0.243207 -0.332798
          04    0.485700 -0.290016 -0.195684
          05   -0.287674  0.574852 -0.287178
          06   -0.084913 -0.071733  0.156646
          07   -0.035495  0.085512 -0.050016
          08    0.147114 -0.007601 -0.139513
          09    0.062996  0.068841 -0.131837
          10   -0.026049  0.174644 -0.148595
          11    0.055942 -0.003382 -0.052560
          12    0.194610 -0.083668 -0.110942
          13    0.025520  0.087642 -0.113162
          14   -0.042680  0.112590 -0.06

## SPR+ data for one station split by ratios

In [67]:
with value(("Bern", "g_220", "Hour")) as (stat, var, scale):
    display(spr_split(stations=stat, variable=var, type="abs", time_scale=scale)[0])
    display(spr_split(stations=stat, variable=var, type="rel", time_scale=scale)[0])

1 Auto  2+ Autos   Keines    Total
DayOfWeek Hour                                     
Monday    00      928.1     182.8    933.4   2044.3
          01        0.0       0.0      0.0      0.0
          02        0.0       0.0      0.0      0.0
          03        0.0       0.0      0.0      0.0
          04        0.0       0.0      0.0      0.0
          05     4444.9     496.7    240.9   5182.5
          06     9743.1    4720.0   5625.6  20088.8
          07    17331.7   15290.3  12613.7  45235.7
          08    17382.7   12365.8  12714.0  42462.5
          09    13094.0    7587.5   7320.8  28002.3
          10     8932.8    7261.9   9104.5  25299.1
          11    11155.1    7971.8   9513.6  28640.5
          12    14825.0    7676.5  14914.1  37415.6
          13    14236.1    6660.5  10958.0  31854.6
          14    14664.5    8290.5   7737.5  30692.6
          15    11885.6    9952.5  12588.6  34426.6
          16    20634.6   17474.4   9848.8  47957.8
          17    30457.0   19317.8  15911.2  65686.0
          18    21823.1   11904.7  20276.4  54004.1
          19    11635.0    7997.2  15526.6  35158.8
          20    11439.7    4988.6   7786.5  24214.8
          21     5178.7    2536.8   9092.8  16808.2
          22     7301.7    1471.2   2591.6  11364.5
          23     4032.5     602.3   1206.4   5841.2
Tuesday   00      423.3     939.7    306.8   1669.8
          01        0.0       0.0      0.0      0.0
          02        0.0       0.0      0.0      0.0
          03        0.0       0.0      0.0      0.0
          04        0.0       0.0      0.0      0.0
          05     3396.6    1074.6    355.2   4826.3
          06     9762.6    4477.2   6257.1  20496.9
          07    18601.9   15090.0  13452.3  47144.2
          08    20616.0    9598.5  13230.5  43445.0
          09    11694.4    7154.0   8517.3  27365.7
          10     7184.9    7619.6   9008.4  23812.9
          11    11544.7   10337.6   5441.3  27323.6
          12    17023.8    9854.0   9306.7  36184.6
          13    10075.8   12872.4   8020.8  30969.1
          14    13640.7    9066.7   6670.5  29377.9
          15    14708.2    9128.5   9810.5  33647.2
          16    21633.0   19557.3   6840.3  48030.5
          17    28695.4   21664.2  16908.3  67267.9
          18    23486.2   14412.2  17888.4  55786.7
          19    11459.4    7204.0  17365.1  36028.5
          20    14159.4    4599.8   6068.3  24827.5
          21     8293.2    4551.0   4695.4  17539.7
          22     6979.2    3460.1   1784.9  12224.2
          23     4051.3    1460.6    935.6   6447.5
Wednesday 00     1158.7     499.6    228.5   1886.8
          01        0.0       0.0      0.0      0.0
          02        0.0       0.0      0.0      0.0
          03        0.0       0.0      0.0      0.0
          04        0.0       0.0      0.0      0.0
          05     2844.5     896.3    921.8   4662.6
          06     8117.8    4405.4   7010.1  19533.3
          07    20232.2   12066.7  12670.0  44968.8
          08    14663.9   12616.7  14276.7  41557.3
          09     8019.7   11019.1   7539.6  26578.4
          10     7128.0    9356.8   7216.9  23701.8
          11     9887.2   10246.5   7515.4  27649.1
          12    13702.6   10404.8  13015.4  37122.7
          13    10709.9   14427.1   6956.3  32093.3
          14    11823.4   10199.0   8420.5  30442.9
          15    15138.5    8247.3  10892.2  34278.0
          16    22710.7   16137.0   8577.4  47425.1
          17    31132.9   19171.4  15318.5  65622.7
          18    19155.9   15791.8  20098.0  55045.6
          19    14905.0    5912.3  15358.8  36176.1
          20    13613.7    4630.8   7431.3  25675.8
          21    11282.4    3634.1   3276.3  18192.8
          22     7776.3    2812.4   2228.1  12816.8
          23     5179.0    1270.8    823.7   7273.6
Thursday  00     1368.8     376.8    444.9   2190.4
          01        0.0       0.0      0.0      0.0
          02        0.0       0.0      0.0      0.0
          03 

1 Auto  2+ Autos  Keines    Total
DayOfWeek Hour                                   
Monday    00      42.1    -394.9   352.8   2044.3
          01      -0.0      -0.0     0.0      0.0
          02       0.0       0.0     0.0      0.0
          03      -0.0       0.0    -0.0      0.0
          04       0.0       0.0     0.0      0.0
          05    2198.7    -967.9 -1230.8   5182.5
          06    1036.4    -957.1   -79.2  20088.8
          07   -2274.1    2506.5  -232.4  45235.7
          08   -1021.1     365.7   655.4  42462.5
          09     957.4    -326.1  -631.3  28002.3
          10   -2032.2     112.2  1920.0  25299.1
          11   -1258.1    -122.1  1380.2  28640.5
          12   -1391.5   -2897.3  4288.8  37415.6
          13     429.8   -2341.7  1911.9  31854.6
          14    1361.9    -383.3  -978.6  30692.6
          15   -3035.5     223.4  2812.0  34426.6
          16    -151.0    3921.3 -3770.3  47957.8
          17    1987.7     754.7 -2742.4  65686.0
          18   -1583.1   -3357.1  4940.2  54004.1
          19   -3603.3   -1938.8  5542.1  35158.8
          20     944.7   -1854.6   909.9  24214.8
          21   -2106.3   -2213.3  4319.5  16808.2
          22    2376.2   -1740.4  -635.7  11364.5
          23    1500.8   -1048.4  -452.4   5841.2
Tuesday   00    -300.4     467.8  -167.4   1669.8
          01      -0.0      -0.0     0.0      0.0
          02       0.0       0.0     0.0      0.0
          03       0.0       0.0     0.0      0.0
          04       0.0       0.0     0.0      0.0
          05    1304.8    -289.4 -1015.4   4826.3
          06     879.0   -1315.3   436.3  20496.9
          07   -1831.1    1766.8    64.2  47144.2
          08    1786.2   -2679.2   893.0  43445.0
          09    -166.3    -579.7   746.0  27365.7
          10   -3136.0     890.0  2246.0  23812.9
          11    -297.7    2615.8 -2318.1  27323.6
          12    1340.9    -371.9  -969.0  36184.6
          13   -3346.7    4120.5  -773.8  30969.1
          14     907.9     764.4 -1672.3  29377.9
          15     125.0    -380.3   255.3  33647.2
          16     815.8    5983.7 -6799.5  48030.5
          17    -459.6    2654.1 -2194.5  67267.9
          18    -692.6   -1353.4  2046.0  55786.7
          19   -4155.9   -2977.8  7133.7  36028.5
          20    3398.8   -2416.5  -982.3  24827.5
          21     691.3    -405.7  -285.5  17539.7
          22    1681.0       5.5 -1686.6  12224.2
          23    1256.9    -361.5  -895.4   6447.5
Wednesday 00     340.9     -33.6  -307.3   1886.8
          01      -0.0       0.0     0.0      0.0
          02       0.0       0.0     0.0      0.0
          03      -0.0      -0.0     0.0      0.0
          04      -0.0      -0.0     0.0      0.0
          05     823.7    -421.4  -402.3   4662.6
          06    -348.2   -1114.8  1463.0  19533.3
          07     742.0    -641.7  -100.3  44968.8
          08   -3347.6     872.4  2475.2  41557.3
          09   -3499.8    3507.9    -8.2  26578.4
          10   -3144.6    2658.6   486.0  23701.8
          11   -2096.3    2432.8  -336.5  27649.1
          12   -2387.0     -86.2  2473.2  37122.7
          13   -3199.8    5357.4 -2157.6  32093.3
          14   -1371.0    1595.7  -224.7  30442.9
          15     281.9   -1439.8  1157.9  34278.0
          16    2156.0    2734.5 -4890.4  47425.1
          17    2691.0     626.2 -3317.2  65622.7
          18   -4701.7     235.7  4466.0  55045.6
          19    -774.3   -4311.2  5085.5  36176.1
          20    2485.4   -2625.2   139.8  25675.8
          21    3397.4   -1507.2 -1890.1  18192.8
          22    2221.3    -809.7 -1411.6  12816.8
          23    2026.5    -784.7 -1241.8   7273.6
Thursday  00     419.4    -242.2  -177.2   2190.4
          01      -0.0       0.0     0.0      0.0
          02       0.0      -0.0    -0.0      0.0
          03       0.0       0.0     0.0      0.0
          04       0.0       0.0     0.0      0.0
          05     679.4   -1122.3   442.9   4648.4
          06    

# Case study: Find best stations / time slots for owner of multiple cars

## Population ratios

In [21]:
ax_population_ratios(variable="g_220", percent=True)

Code,1 Auto,2+ Autos,Keines
Variable,,,
g_220,44,40,16


## Ratios over all stations

In [22]:
ax_global_ratios(variable="g_220", percent=True)

Code,1 Auto,2+ Autos,Keines
Variable,,,
g_220,45,29,26


Multi-car owners are **underrepresented** at digital station panels (29% vs. 40%)

## Ratios per station

In [23]:
ax_station_ratios(variable="g_220", percent=True).actual

Code,1 Auto,2+ Autos,Keines
Station,,,
Aarau,43,30,27
Basel SBB,44,20,36
Bellinzona,52,38,10
Bern,43,28,28
Biel/Bienne,50,25,25
Brig,51,40,9
Chur,48,35,17
Fribourg,31,56,14
Genève Aéroport,41,49,10


Best stations are **Fribourg** (56%), **Zürich Flughafen** (50%), and **Genève Aéroport** (49%).

This compares to 29% over all stations, and 40% in the base population.

## Time slots at one station: Zürich HB

In [29]:
prozent = ax_ratios(
    variable="g_220", stations="Zürich Flughafen", time_scale="Hour", percent=True
).actual
personen = spr_split(
    stations="Zürich HB", variable="g_220", time_scale="Hour", type="abs", decimals=0
)[0]
time_analysis = prozent.merge(
    personen, left_index=True, right_index=True, suffixes=(" [%]", " [Pers.]")
)

time_analysis.loc[prozent["2+ Autos"] > 40]

1 Auto [%]  2+ Autos [%]  Keines [%]  1 Auto [Pers.]  \
DayOfWeek Hour                                                         
Monday    00            48            52           0            1201   
          01             0           100           0             802   
          04            18            82           0               0   
          05            29            60          11            1640   
          07            11            79          10           35382   
          09            38            59           3           14903   
          10            39            61           0           12774   
          11            44            52           4           17373   
          12            28            63           9           26713   
          13            31            67           3           19639   
          14            10            86           5           21581   
          15            34            58           7           25042   
          16            47            50           3           32811   
          19            18            49          33           26588   
          23            22            65          13            5836   
Tuesday   00             0           100           0            1211   
          04             0           100           0             491   
          05             6            79          15             936   
          06            18            62          19           13204   
          07            14            83           3           37496   
          08            33            63           3           31823   
          09            22            62          16           15926   
          10            40            48          12           13403   
          12            36            46          18           24466   
          13            16            72          12           23198   
          14            32            62           6           19592   
          15            38            56           6           18264   
          16            33            61           6           34027   
          18            25            46          30           41999   
          19            27            57          16           25185   
Wednesday 00            49            51           0             940   
          03             0           100           0               0   
          04            42            41          17             603   
          06            25            54          21           12338   
          07            15            65          21           34017   
          08            27            48          25           27954   
          10             7            87           6           15641   
          11            33            50          16           21060   
          12            51            46           3           25098   
          13            29            59          12           17648   
          14            43            52           5           22665   
          15            41            57           3           21336   
          16            41            49          11           30821   
          17            30            45          25           47721   
          18            19            68          14           35651   
          19            25            52          22           24218   
          20            39            46          15           18581   
          21            24            55          21           13117   
          22            42            55           3            8242   
          23            32            42          27            4315   
Thursday  00             0           100           0             710   
          03             0           100           0               0   
          04            31            69           0             597   
          07            40            56           5    

In [30]:
time_analysis.sort_values("2+ Autos [Pers.]", ascending=False).head(20)

1 Auto [%]  2+ Autos [%]  Keines [%]  1 Auto [Pers.]  \
DayOfWeek Hour                                                         
Wednesday 17            30            45          25           47721   
Tuesday   17            54            24          22           49731   
Thursday  16            42            52           6           32081   
          17            43            54           2           50138   
Monday    17            46            40          15           54772   
Tuesday   16            33            61           6           34027   
Wednesday 16            41            49          11           30821   
Friday    17            54            36          10           46920   
Monday    16            47            50           3           32811   
Sunday    16            32            40          28           34851   
Friday    18            45            45          10           33147   
Monday    08            59            28          14           27319   
Wednesday 18            19            68          14           35651   
Tuesday   08            33            63           3           31823   
Wednesday 08            27            48          25           27954   
Sunday    17            41            38          21           31068   
Thursday  08            69            14          17           26439   
Tuesday   18            25            46          30           41999   
Thursday  18            17            64          19           38453   
Friday    16            26            51          23           35824   

                2+ Autos [Pers.]  Keines [Pers.]  Total  
DayOfWeek Hour                                           
Wednesday 17               27631           21283  96636  
Tuesday   17               27317           21822  98870  
Thursday  16               22315           14946  69342  
          17               21937           23174  95249  
Monday    17               21046           20270  96088  
Tuesday   16               20927           15085  70039  
Wednesday 16               20194           18339  69354  
Friday    17               19356           23648  89924  
Monday    16               19282           17564  69656  
Sunday    16               17922           23022  75796  
Friday    18               17786           23776  74708  
Monday    08               17459           13395  58173  
Wednesday 18               17439           26811  79901  
Tuesday   08               16881           10170  58874  
Wednesday 08               16742           11882  56578  
Sunday    17               16693           28748  76509  
Thursday  08               16624           13682  56745  
Tuesday   18               16389           22615  81002  
Thursday  18               16338           24982  79773  
Friday    16               15029           21408  72261

## Time slots with CI at one station: Zürich Flughafen

In [69]:
(counts, sd_ratios) = spr_split(
    stations="Zürich Flughafen", variable="g_220", time_scale="Hour", type="abs", decimals=0
)

counts.merge(
    as_percent(sd_ratios), left_index=True, right_index=True, suffixes=(" [Pers]", " [sd]")
)

1 Auto [Pers]  2+ Autos [Pers]  Keines [Pers]  Total [Pers]  \
DayOfWeek Hour                                                                
Monday    00               11               12              0            24   
          01                0                0              0             0   
          02                0                0              0             0   
          03                0                0              0             0   
          04                0                0              0             0   
          05               17               36              7            59   
          06               86               73             75           233   
          07               57              415             55           527   
          08              290              137             67           494   
          09              124              192             10           326   
          10              115              180              0           295   
          11              146              174             14           334   
          12              122              275             39           436   
          13              115              248             10           372   
          14               35              307             17           358   
          15              138              235             29           401   
          16              266              280             15           561   
          17              349              303            114           766   
          18              197              187            245           630   
          19               74              203            134           410   
          20              136               77             70           283   
          21               69               52             74           195   
          22               14               37             82           133   
          23               15               45              9            69   
Tuesday   00                0               19              0            19   
          01                0                0              0             0   
          02                0                0              0             0   
          03                0                0              0             0   
          04                0                0              0             0   
          05                4               43              8            55   
          06               44              149             46           239   
          07               78              457             17           551   
          08              169              321             17           507   
          09               71              199             50           319   
          10              111              132             35           278   
          11              143              122             54           319   
          12              150              196             76           422   
          13               57              261             42           361   
          14              109              211             22           343   
          15              150              218             25           393   
          16              184              342             35           561   
          17              426              185            173           784   
          18              160              296            194           649   
          19              112              240             68           420   
          20              164               74             51           289   
          21               59               83             63           205   
          22               74               43             25           142   
          23               76                0              0            76   
Wednesday 00               11               11              0     